In [1]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [2]:
ws = Workspace.from_config()
ws

Workspace.create(name='ResNet', subscription_id='92c76a2f-0e1c-4216-b65e-abf7a3f34c1e', resource_group='AzureML_UW_ResNet')

In [4]:
# cluster_name = "ML-CPU-test"
cluster_name = "ML-GPU-test"
compute_target = None

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('No compute cluster named {}'.format(cluster_name))
    exit()

Found existing compute target


In [5]:
curated_env_name = 'Resnet50v15-CPU-cluster'
#pytorch_env = Environment.get(workspace=ws, name=curated_env_name)
pytorch_env = Environment.from_conda_specification(name=curated_env_name, file_path='./conda_dependencies.yml')

In [6]:
from azureml.core import ScriptRunConfig
from azureml.core.datastore import Datastore
from azureml.core import Dataset

project_folder = './'
data_path = 'datasets'

datastore = Datastore.get(ws, 'workspaceblobstore')
dataset = Dataset.File.from_files(path=(datastore, data_path))
data_loc = dataset.as_named_input('input').as_mount()
print(data_loc)

In [ ]:
src = ScriptRunConfig(source_directory=project_folder,
                        # command=['ls'],
                        script='train_resnet.py',
                        arguments=[
                          '--num_epochs', 16,
                          '--batch', '32',
                          '--shuffle', 'True',
                          '--dataloc', data_loc,
                          '--output_dir', './outputs',
                        ],
                        compute_target=compute_target,
                        environment=pytorch_env)

run = Experiment(ws, name='Train-Resnet50v15').submit(src)
run.wait_for_completion(show_output=True)

RunId: Train-Resnet50v15_1623463451_8d32f1a4
Web View: https://ml.azure.com/runs/Train-Resnet50v15_1623463451_8d32f1a4?wsid=/subscriptions/92c76a2f-0e1c-4216-b65e-abf7a3f34c1e/resourcegroups/AzureML_UW_ResNet/workspaces/ResNet&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/55_azureml-execution-tvmps_57529dd1854c626984cc1826ef85efda260a5fc86d3747aba97d8fc455dbe3d2_p.txt

2021-06-12T02:04:27Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root/jobs/resnet/azureml/train-resnet50v15_1623463451_8d32f1a4/mounts/workspaceblobstore
2021-06-12T02:04:28Z Failed to start nvidia-fabricmanager due to exit status 5 with output Failed to start nvidia-fabricmanager.service: Unit nvidia-fabricmanager.service not found.
. Please ignore this if the GPUs don't utilize NVIDIA® NVLink® switches.
2021-06-12T02:04:28Z Starting output-watcher...
2021-06-12T02:04:28Z IsDedicatedCompute == False, starting polling for Low-Pri Preemption
2021-06-12T02:04:28Z Executin